In [1]:
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [2]:
# Load the Data

data_dir = pathlib.Path('./data/tiny-imagenet-200')
image_count = len(list(data_dir.glob('**/*.JPEG')))
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
num_classes = len(CLASS_NAMES)
print('Discovered {} images in {} classes'.format(image_count, num_classes))

# Create the training data generator
batch_size = 32
im_height = 64
im_width = 64
num_epochs = 1

data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
])

# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=data_dir / 'train', transform=data_transforms),
    'valid': datasets.ImageFolder(root=data_dir / 'val', transform=data_transforms),
    'test': datasets.ImageFolder(root=data_dir / 'test', transform=data_transforms)
}

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(data['test'], batch_size=batch_size, shuffle=True)

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

Discovered 129998 images in 200 classes
{0: 'n01443537', 1: 'n01629819', 2: 'n01641577', 3: 'n01644900', 4: 'n01698640', 5: 'n01742172', 6: 'n01768244', 7: 'n01770393', 8: 'n01774384', 9: 'n01774750', 10: 'n01784675', 11: 'n01855672', 12: 'n01882714', 13: 'n01910747', 14: 'n01917289', 15: 'n01944390', 16: 'n01945685', 17: 'n01950731', 18: 'n01983481', 19: 'n01984695', 20: 'n02002724', 21: 'n02056570', 22: 'n02058221', 23: 'n02074367', 24: 'n02085620', 25: 'n02094433', 26: 'n02099601', 27: 'n02099712', 28: 'n02106662', 29: 'n02113799', 30: 'n02123045', 31: 'n02123394', 32: 'n02124075', 33: 'n02125311', 34: 'n02129165', 35: 'n02132136', 36: 'n02165456', 37: 'n02190166', 38: 'n02206856', 39: 'n02226429', 40: 'n02231487', 41: 'n02233338', 42: 'n02236044', 43: 'n02268443', 44: 'n02279972', 45: 'n02281406', 46: 'n02321529', 47: 'n02364673', 48: 'n02395406', 49: 'n02403003', 50: 'n02410509', 51: 'n02415577', 52: 'n02423022', 53: 'n02437312', 54: 'n02480495', 55: 'n02481823', 56: 'n02486410', 

In [3]:
# Load pretrained ResNet50 Model
model = models.resnet50(pretrained=False)

# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

# Convert model to be used on GPU
model = model.to('cuda:0')

# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

best_model_path = "./model_best_resnet50.pth"
# model = torch.load(best_model_path)
state_dict = torch.load(best_model_path)['state_dict']
model.load_state_dict({k[7:]: v for k, v in state_dict.items()})
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
import tsne
all_inputs = []
all_labels = []

for i, (inputs, labels) in enumerate(train_data_loader):
    all_inputs.extend(inputs.to(device))
    all_labels.extend(labels.to(device))
    break

print(all_inputs)
print(all_labels)
    




# # Validation - No gradient tracking needed
# with torch.no_grad():
#     # Set to evaluation mode
#     model.eval()

#     # Validation loop
#     for j, (inputs, labels) in enumerate(data_loader):
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         # Forward pass - compute outputs on input data using the model
#         outputs = model(inputs)
#         loss = loss_criterion(outputs, labels)
#         valid_loss += loss.item() * inputs.size(0)

#         # Calculate validation accuracy
#         ret, predictions = torch.max(outputs.data, 1)
#         correct_counts = predictions.eq(labels.data.view_as(predictions))

#         # Convert correct_counts to float and then compute the mean
#         acc = torch.mean(correct_counts.type(torch.FloatTensor))

#         # Compute total accuracy in the whole batch and add to valid_acc
#         valid_acc += acc.item() * inputs.size(0)

#         #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

#     # Find average training loss and training accuracy
#     avg_valid_loss = valid_loss/valid_data_size 
#     avg_valid_acc = valid_acc/valid_data_size

#     epoch_end = time.time()

#     print("Validation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
#     return avg_valid_acc, avg_valid_loss

[tensor([[[0.0354, 0.0334, 0.0430,  ..., 0.0167, 0.0022, 0.0066],
         [0.0444, 0.0366, 0.0368,  ..., 0.0260, 0.0074, 0.0027],
         [0.0491, 0.0393, 0.0449,  ..., 0.0263, 0.0071, 0.0010],
         ...,
         [0.0513, 0.0577, 0.0457,  ..., 0.0179, 0.0182, 0.0140],
         [0.0361, 0.0481, 0.0464,  ..., 0.0118, 0.0118, 0.0096],
         [0.0278, 0.0432, 0.0467,  ..., 0.0086, 0.0091, 0.0081]],

        [[0.0410, 0.0386, 0.0481,  ..., 0.0177, 0.0037, 0.0083],
         [0.0472, 0.0393, 0.0395,  ..., 0.0273, 0.0088, 0.0044],
         [0.0476, 0.0376, 0.0435,  ..., 0.0282, 0.0096, 0.0034],
         ...,
         [0.0354, 0.0427, 0.0317,  ..., 0.0118, 0.0101, 0.0047],
         [0.0201, 0.0332, 0.0327,  ..., 0.0064, 0.0044, 0.0010],
         [0.0118, 0.0282, 0.0329,  ..., 0.0042, 0.0020, 0.0000]],

        [[0.0042, 0.0064, 0.0233,  ..., 0.0135, 0.0000, 0.0007],
         [0.0172, 0.0135, 0.0204,  ..., 0.0223, 0.0020, 0.0000],
         [0.0270, 0.0204, 0.0317,  ..., 0.0223, 0.0012, 0